# **Machine Learning Model**

This notebook implements a convolutional neural network to recognise different physical activities from Respeck sensor data. The dataset includes multiple 30-second recordings of various physical activities (e.g., ascending stairs, shuffle walking, sitting-standing) stored in separate CSV files for each activity.

This model will be deployed inside the Android app for live classification.


# Imports

In [35]:
# Importing libraries that will be used
import pandas as pd
import numpy as np
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

# Reading Files
Reading files from your dataset

In [13]:
# Path to Respeck data
your_dataset_path = "./PDIoT2324/Respeck/"

This line uses the glob module to find all file paths that match a specified pattern. The 'glob.glob()' function returns a list of file paths that match the given pattern. `your_dataset_path` should be the directory where your dataset files are located.

The `*` is a wildcard character that matches any string of characters,  so this pattern retrieves all folders in the 'your_dataset_path' directory.

In [14]:
glob.glob(your_dataset_path + "*")

['./PDIoT2324/Respeck\\s100_respeck_ascending_breathingNormal.csv',
 './PDIoT2324/Respeck\\s100_respeck_descending_breathingNormal.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_breathingNormal.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_coughing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_hyperventilating.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_laughing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_singing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingBack_talking.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_breathingNormal.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_coughing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_hyperventilating.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_laughing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_singing.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingLeft_talking.csv',
 './PDIoT2324/Respeck\\s100_respeck_lyingRight_breathingNormal.csv',
 './PDIoT2324/Respeck\\s100_respeck_

## Activities and Social Signals Dictionary

In [15]:
# Define activities and social signals with corresponding labels
# Each key is the name of the physical activity, and the corresponding value is the numeric label
# These labels will be used as the target variable for classification
activities_dict = {
    'ascending': 0,
    'shuffleWalking': 1,
    'sittingStanding': 2,
    'miscMovement': 3,
    'normalWalking': 4,
    'lyingBack': 5,
    'lyingLeft': 6,
    'lyingRight': 7,
    'lyingStomach': 8,
    'descending': 9,
    'running': 10
}

social_signals_dict = {
    'breathingNormal': 0,
    'coughing': 1,
    'hyperventilating': 2,
    'other': 3
}

# Functions

## Load list of files in an activity folder

In [16]:
def load_files_from_folder(folder_path):
    """
    Load all CSV files from a folder, extract activity and social signal information,
    and return a list of file paths along with combined labels and file information.

    Parameters:
    folder_path (str): The path to the folder containing CSV files.

    Returns:
    tuple: A tuple containing:
        - list: A list of file paths for all CSV files in the folder.
        - list: A list of combined labels (activity and social signal) for each file.
        - dict: A dictionary containing file information with activity and social signal labels.
    """

    # Initialise an empty list to store the full file paths of the CSV files
    file_paths = []

    # Initialise an empty dictionary to store the filenames and the activity and social signal labels
    file_info = {}

    # Initialise an empty list to store the combined labels of the activity and social signal to be used in stratified split of data
    # Stratified split of data ensures that each activity-social signal combination is proportionally represented in the training and testing data
    combined_labels = []

    # Loop through all the files in the given folder
    for file_name in os.listdir(folder_path):
        
        # Check if the file has a .csv extension (ignores other files)
        if file_name.endswith('.csv'):

            # Inialise an empty dictionary for each file to store the activity and social signal labels
            file_info[file_name] = {}

            # Split the file name by underscores to extract activity and social signal information
            parts = file_name.split("_")
            
            # Extract the activity from the file name
            activity = parts[2]

            # Add activity label to the file_info dictionary
            if activity == "sitting" or activity == "standing":
                file_info[file_name]['activity_label'] = activities_dict["sittingStanding"]
            else:
                file_info[file_name]['activity_label'] = activities_dict[activity]
            
            # Extract the social signal from the file name, without the .csv extension
            social_signal = parts[3].split(".")[0]

            # Add social signal label to the file_info dictionary
            if social_signal == "laughing" or social_signal == "eating" or social_signal == "talking" or social_signal == "singing":
                file_info[file_name]['social_signal_label'] = social_signals_dict["other"]
            else:
                file_info[file_name]['social_signal_label'] = social_signals_dict[social_signal]
            
            # Combine the activity and social signal to create a unique label
            combined_label = activity + "_" + social_signal

            # Append the combined label to the combined_labels list
            combined_labels.append(combined_label)

            # Construct the full file path by joining the folder path and the file name'
            full_file_path = os.path.join(folder_path, file_name)

            # Append the full file path to the file_paths list
            file_paths.append(full_file_path)

    # Return the complete list of CSV file paths
    return file_paths, combined_labels, file_info

## Train and test set split from list of files

In [17]:
def split_files(file_list, combined_labels, test_size=0.2):
    """
    Split the list of file paths into training and testing sets.

    Parameters:
    file_paths (list): A list of file paths for all CSV files.
    combined_labels (list): A list of combined labels (activity and social signal) for each file.
    test_size (float, optional): The proportion of the dataset to include in the test split. Default is 0.2.

    Returns:
    tuple: A tuple containing:
        - list: Training file paths.
        - list: Testing file paths.
    """

    # Split the file list into training and test sets using train_test_split from scikit-learn
    # test_size defines the proportion of the data to use as the test set (default is 20%)
    # shuffle=True ensures that the files are shuffled randomly before splitting
    # stratify is used to ensure that the proportion of each class is the same in both the training and testing sets
    train_files, test_files = train_test_split(file_list, test_size=test_size, stratify = combined_labels, shuffle=True, random_state=42)

    # Return the train and test file lists
    return train_files, test_files

## Sliding Window

In time series Activity Recognition, a sliding window is a commonly used technique to segment continuous sensor data (such as accelerometer readings) into smaller, fixed-length overlapping or non-overlapping time intervals, or windows. Each window contains a sequence of sensor measurements that represent a short period of time, and this segmented data is used to extract features or make predictions about the activity happening within that window.

### Key Concepts of a Sliding Window
1.   **Window Size:** This refers to the length of each segment or window, typically defined in terms of the number of time steps or the duration (e.g., 2 seconds). The window size should be chosen carefully to capture enough information about the activity without making the window too large.
2.   **Step Size:** The step size determines how far the window moves forward after each step. If the step size is smaller than the window size, the windows will overlap. For example, if the window size is 5 seconds and the step size is 2 seconds, there will be a 3-second overlap between consecutive windows. Overlapping windows provide more data for analysis and can help smooth out predictions by capturing transitional activities.
3.   **Non-Overlapping Windows:** If the step size is equal to the window size, the windows do not overlap. This method provides distinct segments of data but may miss transitional phases between activities.

### Why Sliding Windows for Activity Recognition?

* Segmentation of Continuous Data: Activity recognition systems work with continuous streams of sensor data, and the sliding window helps segment these into manageable pieces to classify activities within specific intervals.

* Context Capturing: Human activities are often complex and spread across time. By using a sliding window, you can capture context across a short duration, which may include transitions or small fluctuations in the activity (e.g., a person moving from sitting to standing).

* Feature Extraction: Within each window, features such as mean, variance, frequency domain features, etc., can be extracted to help classify the activity.

* Real-Time Recognition: In real-time systems, the sliding window allows for continuous monitoring and updating of predictions as new data arrives.



In [18]:
def load_and_apply_sliding_windows(file_path, window_size, step_size, file_info):
    """
    Load the data from each file, apply sliding windows, and return the windows and labels.

    Parameters:
    file_paths (list): List of file paths to CSV files. Each file contains sensor data.
    window_size (int): The size of each sliding window (number of time steps).
    step_size (int): The step size (stride) between consecutive windows.
    file_info (dict): Dictionary containing file information with activity and social signal labels.

    Returns:
    tuple:
        - windows (numpy.ndarray): A 3D array of sliding windows, where each window has the shape
                                   (num_windows, window_size, num_features).
        - activity_labels (numpy.ndarray): A 1D array of activity labels, where each label corresponds to a sliding window.
        - social_signal_labels (numpy.ndarray): A 1D array of social signal labels, where each label corresponds to a sliding window.
    """
    # Initialise lists to store sliding windows and their corresponding labels
    windows = []
    activity_labels = []
    social_signal_labels = []
    file_number = 0

    
    # Loop through each file in the provided file path
    for file in file_path:

        # Extract the activity and social signal labels from the file_info dictionary
        activity_label = file_info[os.path.basename(file)]['activity_label']
        social_signal_label = file_info[os.path.basename(file)]['social_signal_label']

        # Load the CSV file into a pandas DataFrame
        data = pd.read_csv(file)   


        # Select the columns containing the necessary sensor data (acceleration readings)
        # These columns might vary depending on your dataset's structure
        data = data[['accel_x', 'accel_y', 'accel_z']]
        
        # Convert the DataFrame into a numpy array for faster processing in the sliding window operation
        data = data.to_numpy()

        
        # Get the number of samples (rows) and features (columns) in the data
        num_samples, num_features = data.shape
        
        # Apply sliding windows to the data
        # The range function defines the start of each window, moving step_size increments at a time
        for i in range(0, num_samples - window_size + 1, step_size):
            # Extract a window of size 'window_size' from the current position 'i'
            window = data[i:i + window_size, :]

            # Append the window to the windows list
            windows.append(window)

            # Assign the activity label to the window and append it to the activity labels list
            activity_labels.append(activity_label)

            # Assign the social signal label to the window and append it to the social signal labels list
            social_signal_labels.append(social_signal_label)

    # Convert the lists of windows and labels into numpy arrays for efficient numerical operations
    return np.array(windows), np.array(activity_labels), np.array(social_signal_labels) 

    
    

## Load and Split Train Test for Each Activity Folder

This function processes the sensor data for a specific activity, such as 'walking' or 'running', stored in its respective folder. It splits the data into training and testing sets, applies sliding windows, and labels the windows with the corresponding activity. This function can be used repeatedly for each activity to process and prepare data for training and evaluation.

In [21]:
def process_activity(dataset_path, window_size=50, step_size=50, test_size=0.2):
    """
    Processes an activity folder by loading the file list, splitting them into
    train and test sets, and applying sliding windows to the files.

    Args:
        dataset_path (str): Base path where the activity folders are located.
        window_size (int): Size of the sliding window, i.e., the number of time steps included in each window.
                           Default is 50.
        step_size (int): Step size for the sliding window, i.e., how far the window moves along the data.
                         Default is 50 (no overlap between windows).
        test_size (float): Proportion of files to use for testing. Default is 0.2, meaning 20% of files will
                           be allocated to the test set.

    Returns:
        tuple:
            - train_windows (numpy.ndarray): Sliding windows from the training files.
            - train_activity_labels (numpy.ndarray): Corresponding activity labels for the training windows.
            - train_social_signal_labels (numpy.ndarray): Corresponding social signal labels for the training windows.
            - test_windows (numpy.ndarray): Sliding windows from the test files.
            - test_activity_labels (numpy.ndarray): Corresponding activity labels for the test windows.
            - test_social_signal_labels (numpy.ndarray): Corresponding social signal labels for the test windows.
    """

    # Load all CSV file paths for the given activity from the folder
    file_paths, combined_labels, file_info = load_files_from_folder(dataset_path)

    # Split the file list into training and testing sets
    # train_files: files used for training
    # test_files: files used for testing
    train_files, test_files = split_files(file_paths, combined_labels, test_size=test_size)

    # Apply sliding windows to the training files
    # The function 'load_and_apply_sliding_windows' returns the sliding windows (segments) and their corresponding activity and social signal labels
    train_windows, train_activity_labels, train_social_signal_labels = load_and_apply_sliding_windows(train_files, window_size, step_size, file_info)

    # Apply sliding windows to the testing files
    test_windows, test_activity_labels, test_social_signal_labels = load_and_apply_sliding_windows(test_files, window_size, step_size, file_info)

    # Return the sliding windows and their labels for both training and testing sets
    return train_windows, train_activity_labels, train_social_signal_labels, test_windows, test_activity_labels, test_social_signal_labels

## 1D CNN Model

This function, `build_1d_cnn_model`, creates and compiles a 1D Convolutional Neural Network (CNN) for multi-label classification tasks.

### Function Overview

Input Parameters
* `input_shape`: Specifies the shape of the input data. It represents (timesteps, features), where timesteps refer to the length of the time series (e.g., 50 windows), and features represent the number of measurements in each time step (e.g., accelerometer readings).
* `num_activity_classes`: The number of output classes for the activity classification problem.
* `num_social_signal_classes`: The number of output classes for the social signal classification problem.

Returns
* The function returns a compiled 1D CNN model with two outputs that is ready to be trained on your data.

<hr>

### Function Breakdown
1. **Model Initialization:**
    * `inputs = Input(shape=input_shape)`: Initializes the input layer with the specified shape.
2. **First Convolutional Layer:**
    * `Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)`
        * This is the first 1D convolutional layer.
        * `filters=64`: The layer applies 64 filters (or kernels) over the input data.
        * `kernel_size=3`: Each filter will cover 3 timesteps at a time (a window of 3).
        * `activation='relu'`: The Rectified Linear Unit (ReLU) activation function introduces non-linearity and helps the model learn complex patterns.
    * `MaxPooling1D(pool_size=2)(x)`: This pooling layer reduces the dimensionality of the data by taking the maximum value from each 2-timestep window (`pool_size=2`).
3. **Second Convolutional Layer:**
    * `Conv1D(filters=128, kernel_size=3, activation='relu')(x)`
        * This is the second convolutional layer, similar to the first, but with 128 filters.
        * `kernel_size=3` and `activation='relu'` function in the same way as the first Conv1D layer.
    * `MaxPooling1D(pool_size=2)(x)`: Another pooling layer to downsample the output, further reducing the data’s dimensionality.
4. **Flattening Layer:**
    * `Flatten()(x)`: Converts the 2D output of the convolutional and pooling layers into a 1D vector.
5. **Fully Connected Layer:**
    * `Dense(128, activation='relu')(x)`: This is a fully connected layer with 128 units/neurons.
6. **Dropout Layer:**
    * `Dropout(0.5)(x)`: This layer randomly sets 50% of the neurons to zero during training to prevent overfitting.
7. **Output Layer for Activity Classification:**
    * `Dense(num_activity_classes, activation='softmax', name='activity_output')(x)`: This is the output layer for activity classification with `num_activity_classes` neurons.
8. **Output Layer for Social Signal Classification:**
    * `Dense(num_social_signal_classes, activation='softmax', name='social_signal_output')(x)`: This is the output layer for social signal classification with `num_social_signal_classes` neurons.
9. **Model Definition:**
    * `model = Model(inputs=inputs, outputs=[activity_output, social_signal_output])`: Defines the model with two outputs.
10. **Compiling the Model:**
    * `model.compile(optimizer='adam', loss={'activity_output': 'categorical_crossentropy', 'social_signal_output': 'categorical_crossentropy'}, metrics={'activity_output': 'accuracy', 'social_signal_output': 'accuracy'})`
        * Optimizer: 'adam': Adam is an optimization algorithm that adjusts the learning rate during training to improve performance.
        * Loss: 'categorical_crossentropy': This loss function is used for multi-class classification problems where the target variable is one-hot encoded.
        * Metrics: ['accuracy']: The accuracy metric is used to evaluate the model’s performance during training and testing.

In [24]:
def build_1d_cnn_model(input_shape, num_activity_classes, num_social_signal_classes):
    """
    Builds and compiles a 1D CNN model for multi-label classification.

    Args:
        input_shape (tuple): The shape of the input data (timesteps, features).
        num_activity_classes (int): The number of output activity classes.
        num_social_signal_classes (int): The number of output social signal classes.

    Returns:
        model (Sequential): Compiled 1D CNN model with two outputs.
    """
    # Input layer
    inputs = Input(shape=input_shape)

    # First Conv1D layer
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    x = MaxPooling1D(pool_size=2)(x)

    # Second Conv1D layer
    x = Conv1D(filters=128, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # Flatten the output from the convolutional layers
    x = Flatten()(x)

    # Fully connected layer
    x = Dense(128, activation='relu')(x)

    # Dropout layer for regularization
    x = Dropout(0.5)(x)

    # Output layer for activity classification
    activity_output = Dense(num_activity_classes, activation='softmax', name='activity_output')(x)

    # Output layer for social signal classification
    social_signal_output = Dense(num_social_signal_classes, activation='softmax', name='social_signal_output')(x)

    # Define the model with two outputs
    model = Model(inputs=inputs, outputs=[activity_output, social_signal_output])

    # Compile the model with two separate losses
    model.compile(optimizer='adam',
                  loss={'activity_output': 'categorical_crossentropy', 'social_signal_output': 'categorical_crossentropy'},
                  metrics={'activity_output': 'accuracy', 'social_signal_output': 'accuracy'})

    # Print a detailed summary of the model
    model.summary()

    return model

# Classification Pipeline

## Step 1: Prepare and Preprocess the Data

Now the training and testing data will be created by calling the function `process_activity`. The `process_activity` function is used to generate sliding windows and labels for the training and testing sets.
* `X_train` and `X_test` are 3D arrays of sliding windows (shape: num_windows, window_size, num_features).
* `y_train_activity` and `y_train_social_signal` are 1D arrays of activity and social signal labels for the training set.
* `y_test_activity` and `y_test_social_signal` are 1D arrays of activity and social signal labels for the testing set.

In [26]:
# Generate the sliding windows along with activity and social signal labels for training and testing sets
X_train, y_train_activity, y_train_social_signal, X_test, y_test_activity, y_test_social_signal = process_activity(your_dataset_path, test_size=0.2, window_size=50, step_size=50)
print(f"{len(X_train)} train windows generated with {len(y_train_activity)} activity labels and {len(y_train_social_signal)} social signal labels")
print(f"{len(X_test)} test windows generated with {len(y_test_activity)} activity labels and {len(y_test_social_signal)} social signal labels")


48575 train windows generated with 48575 activity labels and 48575 social signal labels
12154 test windows generated with 12154 activity labels and 12154 social signal labels


Now the training and testing sets generated by the `process_activity` function are checked to see that they have the correct shape.

In [27]:
# Print the shapes of the training and test arrays to verify that everything has been combined correctly
print(f"X_train shape: {X_train.shape}, y_train_activity shape: {y_train_activity.shape}, y_train_social_signal shape: {y_train_social_signal.shape}")
print(f"X_test shape: {X_test.shape}, y_test_activity shape: {y_test_activity.shape}, y_test_social_signal shape: {y_test_social_signal.shape}")

X_train shape: (48575, 50, 3), y_train_activity shape: (48575,), y_train_social_signal shape: (48575,)
X_test shape: (12154, 50, 3), y_test_activity shape: (12154,), y_test_social_signal shape: (12154,)


### One-Hot Encode Labels (for multi-class classification)
Since there are more than two classes, the labels must be one-hot encoded, especially as the model will use categorical cross-entropy loss.

One-Hot Encoding converts categorical labels into binary vectors (one-hot encoded format). Each class label is represented as a binary vector with 1 for the correct class and 0 for others. This is necessary for training models that use categorical_crossentropy as the loss function, such as a neural network.

In [29]:
# Initialise the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit the OneHotEncoder on the training activity labels and transform them to one-hot encoded format
y_train_activity_one_hot = encoder.fit_transform(y_train_activity.reshape(-1, 1))

# Transform the test activity labels to one-hot encoded format using the already fitted encoder
y_test_activity_one_hot = encoder.transform(y_test_activity.reshape(-1, 1))


# Fit the OneHotEncoder on the training social signal labels and transform them to one-hot encoded format
y_train_social_signal_one_hot = encoder.fit_transform(y_train_social_signal.reshape(-1, 1))

# Transform the test social signal labels to one-hot encoded format using the already fitted encoder
y_test_social_signal_one_hot = encoder.transform(y_test_social_signal.reshape(-1, 1))

# Explanation:
# - y_train_activity_one_hot, y_train_social_signal_one_hot, y_test_activity_one_hot and y_test_social_signal_one_hot are now 2D arrays where each row is a one-hot encoded binary vector corresponding to a class label.
# - The number of columns in the one-hot encoded labels equals the number of unique classes (activities).
# For example, if there are 6 unique activities, the encoded vector will have 6 elements, with a '1' indicating the correct class.

In [30]:
# Print the shapes of the one-hot encoded labels to verify that the transformation was successful
print(f"y_train_activity_one_hot shape: {y_train_activity_one_hot.shape}, y_train_social_signal_one_hot shape: {y_train_social_signal_one_hot.shape}, y_test_activity_one_hot shape: {y_test_activity_one_hot.shape}, y_test_social_signal_one_hot shape: {y_test_social_signal_one_hot.shape}")

# Explanation of shapes:
# - The shape of y_train_activity_one_hot will be (num_samples, num_classes), where:
#     - num_samples is the number of training windows.
#     - num_classes is the number of unique activities (the length of the one-hot vectors).
# - Similarly, y_test_activity_one_hot will have the same number of columns (num_classes) as y_train_activity_one_hot but will have fewer rows (corresponding to the number of test windows).

y_train_activity_one_hot shape: (48575, 11), y_train_social_signal_one_hot shape: (48575, 4), y_test_activity_one_hot shape: (12154, 11), y_test_social_signal_one_hot shape: (12154, 4)


## Step 2: Build the 1D-CNN Model
Call our `build_1d_cnn_model` function to build our model

In [31]:
# Determine the input shape for the model
input_shape = (X_train.shape[1], X_train.shape[2])

# Determine the number of output classes (num_classes)
num_activity_classes = y_train_activity_one_hot.shape[1]
num_social_signal_classes = y_train_social_signal_one_hot.shape[1]

# Build and compile the model
# The function will return a compiled model ready for training
model = build_1d_cnn_model(input_shape, num_activity_classes, num_social_signal_classes)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 50, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 48, 64)    │        640 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 24, 64)    │          0 │ conv1d[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 22, 128)   │     24,704 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 11, 128)   │          0 │ conv1d_1[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 1408)      │          0 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    180,352 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activity_output     │ (None, 11)        │      1,419 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ social_signal_outp… │ (None, 4)         │        516 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 207,631 (811.06 KB)

 Trainable params: 207,631 (811.06 KB)

 Non-trainable params: 0 (0.00 B)

## Step 3: Train the CNN Model

Train the 1D CNN model using the training data and validate on the test data. The model will learn to map input sliding windows to their corresponding activity and social signal labels.

`model.fit()` is used to train the neural network model. It takes several parameters:
* `X_train`: The input training data (sliding windows), with shape (num_samples, window_size, num_features).
* `{'activity_output': y_train_activity_one_hot, 'social_signal_output': y_train_social_signal_one_hot}`: The corresponding one-hot encoded labels for the training data, with shape (num_samples, num_classes).
* `epochs`: Number of times the entire training dataset is passed through the model. In this case, we are training for 20 epochs, meaning the model will see the entire training set 20 times.
* `batch_size`: Number of samples processed before the model's weights are updated. Here, the batch size is set to 32, meaning the model will process 32 samples at a time before updating its parameters.
* `validation_data`: This parameter allows us to evaluate the model's performance on the test data after each epoch. It takes the test data and corresponding one-hot encoded test labels.

In [32]:
history = model.fit(
    X_train,  # Input data
    {'activity_output': y_train_activity_one_hot, 'social_signal_output': y_train_social_signal_one_hot},  # Target labels
    epochs=20, # Train the model for 20 epochs
    batch_size=32, # Use a batch size of 32
    validation_data=(X_test, {'activity_output': y_test_activity_one_hot, 'social_signal_output': y_test_social_signal_one_hot}) # Validate the test set after each epoch
)


Epoch 1/20
1518/1518 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - activity_output_accuracy: 0.8688 - activity_output_loss: 0.5125 - loss: 1.6992 - social_signal_output_accuracy: 0.5182 - social_signal_output_loss: 1.1867 - val_activity_output_accuracy: 0.9084 - val_activity_output_loss: 0.2955 - val_loss: 1.3331 - val_social_signal_output_accuracy: 0.5863 - val_social_signal_output_loss: 1.0371
Epoch 2/20
1518/1518 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - activity_output_accuracy: 0.9165 - activity_output_loss: 0.2625 - loss: 1.2478 - social_signal_output_accuracy: 0.5904 - social_signal_output_loss: 0.9854 - val_activity_output_accuracy: 0.9264 - val_activity_output_loss: 0.2580 - val_loss: 1.1815 - val_social_signal_output_accuracy: 0.6052 - val_social_signal_output_loss: 0.9230
Epoch 3/20
1518/1518 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - activity_output_accuracy: 0.9312 - activity_output_loss: 0.2186 - loss: 1.1519 - social_signal_output_accuracy: 0.5971 - social_signal_output_loss: 0.9333 - val_activit

`val_accuracy` is the accuracy of the model on the validation data (in this case X_test, y_test_activity_one_hot and y_test_social_signal_one_hot). The `accuracy` is the training accuracy for the current epoch.

## Step 4: Evaluate the Model
After training, the model is evaluated on the test set. This is done with a classification report and 5-Fold Cross-Validation.

### Classification report

In [ ]:
# Get predicted probabilities for the test set
y_pred_probs = model.predict(X_test)

# Seperate the predictions for activity and social signal
y_pred_probs_activity = y_pred_probs[0]
y_pred_probs_social_signal = y_pred_probs[1]

# Convert the predicted probabilities to class labels (taking the argmax of the probabilities)
y_pred_classes_activity = np.argmax(y_pred_probs_activity, axis=1)
y_pred_classes_social_signal = np.argmax(y_pred_probs_social_signal, axis=1)

# Convert the true test labels from one-hot encoding back to class labels
y_true_classes_activity = np.argmax(y_test_activity_one_hot, axis=1)
y_true_classes_social_signal = np.argmax(y_test_social_signal_oane_hot, axis=1)

# Generate the classification report for activity labels
report_activity = classification_report(y_true_classes_activity, y_pred_classes_activity, digits=4)
print("Classification Report for Activity Labels:")
print(report_activity)

# Generate the classification report for social signal labels
report_social_signal = classification_report(y_true_classes_social_signal, y_pred_classes_social_signal, digits=4)
print("Classification Report for Social Signal Labels:")
print(report_social_signal)

380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Classification Report for Activity Labels:
              precision    recall  f1-score   support

           0     0.9036    0.8333    0.8671       270
           1     0.7552    0.5370    0.6277       270
           2     0.9737    0.9923    0.9829      3887
           3     0.6458    0.6889    0.6667       270
           4     0.7348    0.8099    0.7705       284
           5     0.9765    1.0000    0.9881      1659
           6     0.9898    0.9946    0.9922      1660
           7     0.9975    0.9729    0.9850      1658
           8     0.9868    0.9897    0.9883      1658
           9     0.8125    0.8246    0.8185       268
          10     0.9741    0.8370    0.9004       270

    accuracy                         0.9589     12154
   macro avg     0.8864    0.8618    0.8716     12154
weighted avg     0.9585    0.9589    0.9580     12154

Classification Report for Social Signal Labels:
              precision    recall  f1-score   support



### 5-Fold Cross-Validation

In [36]:
# Set up KFold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialise lists to store the accuracy of each fold
activity_accuracy = []
social_signal_accuracy = []

# Perform KFold cross-validation
fold_no = 1

for train_index, val_index in kfold.split(X_train):
    print(f"Training fold {fold_no}")

    # Split the data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_activity_fold, y_val_activity_fold = y_train_activity_one_hot[train_index], y_train_activity_one_hot[val_index]
    y_train_social_signal_fold, y_val_social_signal_fold = y_train_social_signal_one_hot[train_index], y_train_social_signal_one_hot[val_index]

    # Build and compile the model
    model = build_1d_cnn_model(input_shape, num_activity_classes, num_social_signal_classes)

    # Train the model on this fold
    history = model.fit(
        X_train_fold,
        {'activity_output': y_train_activity_fold, 'social_signal_output': y_train_social_signal_fold},
        epochs=20,
        batch_size=32,
        validation_data=(X_val_fold, {'activity_output': y_val_activity_fold, 'social_signal_output': y_val_social_signal_fold})
    )

    # Evaluate the model on the validation set
    y_pred_probs = model.predict(X_val_fold)
    y_pred_probs_activity = np.argmax(y_pred_probs[0], axis = 1)
    y_pred_probs_social_signal = np.argmax(y_pred_probs[1], axis = 1)

    y_true_activity = np.argmax(y_val_activity_fold, axis = 1)
    y_true_social_signal = np.argmax(y_val_social_signal_fold, axis = 1)

    # Generate the classification report for activity and social signal labels
    report_activity = classification_report(y_true_activity, y_pred_probs_activity, output_dict=True)
    report_social_signal = classification_report(y_true_social_signal, y_pred_probs_social_signal, output_dict=True)

    # Append the reports to the lists for averaging later
    activity_accuracy.append(report_activity['accuracy'])
    social_signal_accuracy.append(report_social_signal['accuracy'])

    print(f"Completed fold {fold_no}")
    fold_no += 1

average_activity_accuracy = np.mean(activity_accuracy)
average_social_signal_accuracy = np.mean(social_signal_accuracy)

print(f"Average Activity Accuracy: {average_activity_accuracy:.4f}")
print(f"Average Social Signal Accuracy: {average_social_signal_accuracy:.4f}")


Training fold 1


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 50, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 48, 64)    │        640 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 24, 64)    │          0 │ conv1d_2[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 22, 128)   │     24,704 │ max_pooling1d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_3     │ (None, 11, 128)   │          0 │ conv1d_3[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 1408)      │          0 │ max_pooling1d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │    180,352 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activity_output     │ (None, 11)        │      1,419 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ social_signal_outp… │ (None, 4)         │        516 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 207,631 (811.06 KB)

 Trainable params: 207,631 (811.06 KB)

...

 Total params: 207,631 (811.06 KB)

 Trainable params: 207,631 (811.06 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
1215/1215 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - activity_output_accuracy: 0.8630 - activity_output_loss: 0.5369 - loss: 1.7344 - social_signal_output_accuracy: 0.5158 - social_signal_output_loss: 1.1975 - val_activity_output_accuracy: 0.9142 - val_activity_output_loss: 0.2605 - val_loss: 1.3060 - val_social_signal_output_accuracy: 0.5545 - val_social_signal_output_loss: 1.0450
Epoch 2/20
1215/1215 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - activity_output_accuracy: 0.9152 - activity_output_loss: 0.2724 - loss: 1.2678 - social_signal_output_accuracy: 0.5885 - social_signal_output_loss: 0.9955 - val_activity_output_accuracy: 0.9335 - val_activity_output_loss: 0.2139 - val_loss: 1.1350 - val_social_signal_output_accuracy: 0.5950 - val_social_signal_output_loss: 0.9206
Epoch 3/20
 244/1215 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - activity_output_accuracy: 0.9319 - activity_output_loss: 0.2143 - loss: 1.1412 - social_signal_output_accuracy: 0.5987 - social_signal_output_loss: 0.9269

KeyboardInterrupt: 

# Exporting your model to TFLite

You can use the TFLiteConverter class provided by TensorFlow to convert your trained model into the TensorFlow Lite format. We export models to TensorFlow Lite (TFLite) for several reasons, primarily because TFLite is designed for deployment on edge devices, such as mobile phones, embedded systems, IoT devices, and microcontrollers, where computational resources and power are limited. This is necessary as you will be running your ML models on your Android devices to perform live classification.

In [24]:
# Convert the trained Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # model is your trained Keras model
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully exported to model.tflite")

INFO:tensorflow:Assets written to: C:\Users\seanc\AppData\Local\Temp\tmpcjr4lwin\assets


INFO:tensorflow:Assets written to: C:\Users\seanc\AppData\Local\Temp\tmpcjr4lwin\assets


Saved artifact at 'C:\Users\seanc\AppData\Local\Temp\tmpcjr4lwin'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  List[TensorSpec(shape=(None, 11), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)]
Captures:
  1387511079824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1387511079632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1387511082128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1387511082320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1387511080976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1387511082896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1387511083472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1387511085584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1387511083088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1387511084048: